In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_btc = spark.table("btc_m2.bronze.btc_raw")
df_liq = spark.table("btc_m2.bronze.liquidity_raw")

df_btc = df_btc.withColumn("data_join", F.to_date(F.col("Date")))
df_liq = df_liq.withColumn("data_join", F.to_date(F.col("date")))

df_joined = df_btc.join(df_liq, on="data_join", how="left")

window_spec = Window.orderBy("data_join").rowsBetween(Window.unboundedPreceding, 0)

df_filled = df_joined.withColumn(
    "liquidez_global_ffill",
    F.last("m2_supply", ignorenulls=True).over(window_spec)
)

df_silver = df_filled.select(
    F.col("data_join").alias("data"),
    F.col("Close").alias("preco_btc"),
    F.col("liquidez_global_ffill").alias("liquidez_global")
).orderBy("data")

df_silver = df_silver.filter(F.col("liquidez_global").isNotNull())

df_silver.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("btc_m2.silver.consolidado")

display(df_silver)